In [1]:
from flytracker.videoreader import VideoReader
from flytracker.preprocessing import preprocessing
import cv2 as cv 
import torch
from flytracker.utils import FourArenasQRCodeMask
from torch.utils.data import DataLoader
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
torch.cuda.is_available()

True

In [3]:
class SingleThreadVideoDataset(torch.utils.data.IterableDataset):
    def __init__(self, path, mask):
        super().__init__()
        self.capture = cv.VideoCapture(path)
        self.mask = torch.tensor(mask, dtype=torch.bool)

    def __iter__(self):
        return self

    def __next__(self) -> torch.Tensor:
        succes, image = self.capture.read()
        if succes is False:
            raise StopIteration
        # because we use opencv
        image = np.moveaxis(image, -1, 0)
        processed_image = preprocessing(image, self.mask)
        return processed_image

In [4]:
class MultiThreadVideoDataset(torch.utils.data.IterableDataset):
    def __init__(self, path, mask):
        super().__init__()
        self.reader = VideoReader(path, max_queue=100)
        self.mask = torch.tensor(mask, dtype=torch.bool)

    def __iter__(self):
        return self

    def __next__(self) -> torch.Tensor:
        succes, image = self.reader.read()
        if succes is False:
            raise StopIteration
        # because we use opencv
        image = np.moveaxis(image, -1, 0)
        processed_image = preprocessing(image, self.mask)
        return processed_image

In [5]:
mask = FourArenasQRCodeMask().mask
path = "/home/gert-jan/Documents/flyTracker/data/movies/4arenas_QR.h264"

In [8]:
%%time
dataset = SingleThreadVideoDataset(path, mask)
loader = DataLoader(dataset, batch_size=1, pin_memory=True)
for batch_idx, batch in enumerate(loader):
    batch = batch.cuda(non_blocking=True)
    if batch_idx % 100 == 0:
        print(f"Loaded {batch_idx}, {batch.device}")
        print(torch.mean(batch.type(torch.float32)))
    if batch_idx == 1000:
        break

Loaded 0, cuda:0
tensor(235.7776, device='cuda:0')
Loaded 100, cuda:0
tensor(235.8470, device='cuda:0')
Loaded 200, cuda:0
tensor(235.7998, device='cuda:0')
Loaded 300, cuda:0
tensor(235.8143, device='cuda:0')
Loaded 400, cuda:0
tensor(235.8383, device='cuda:0')
Loaded 500, cuda:0
tensor(235.8459, device='cuda:0')
Loaded 600, cuda:0
tensor(235.7329, device='cuda:0')
Loaded 700, cuda:0
tensor(235.8555, device='cuda:0')
Loaded 800, cuda:0
tensor(235.8855, device='cuda:0')
Loaded 900, cuda:0
tensor(235.7736, device='cuda:0')
Loaded 1000, cuda:0
tensor(235.8390, device='cuda:0')
CPU times: user 57.2 s, sys: 880 ms, total: 58.1 s
Wall time: 3.41 s


In [7]:
%%time
dataset = MultiThreadVideoDataset(path, mask)
loader = DataLoader(dataset, batch_size=1, pin_memory=True)
for batch_idx, batch in enumerate(loader):
    batch = batch.cuda(non_blocking=True)
    if batch_idx % 100 == 0:
        print(f"Loaded {batch_idx}, {batch.device}")
        print(torch.mean(batch.type(torch.float32)))
    if batch_idx == 1000:
        break
loader.dataset.reader.stop()

Loaded 0, cuda:0
tensor(235.7776, device='cuda:0')
Loaded 100, cuda:0
tensor(235.8470, device='cuda:0')
Loaded 200, cuda:0
tensor(235.7998, device='cuda:0')
Loaded 300, cuda:0
tensor(235.8143, device='cuda:0')
Loaded 400, cuda:0
tensor(235.8383, device='cuda:0')
Loaded 500, cuda:0
tensor(235.8459, device='cuda:0')
Loaded 600, cuda:0
tensor(235.7329, device='cuda:0')
Loaded 700, cuda:0
tensor(235.8555, device='cuda:0')
Loaded 800, cuda:0
tensor(235.8855, device='cuda:0')
Loaded 900, cuda:0
tensor(235.7736, device='cuda:0')
Loaded 1000, cuda:0
tensor(235.8390, device='cuda:0')
CPU times: user 41 s, sys: 523 ms, total: 41.5 s
Wall time: 2.22 s
